# Impact on Convergence Time of Swarmalators by using Momentum
Simulation<br>
D. Kofler, 2022

This simulation was initially created based on the paper 'Swarmalators with Stochastic Coupling and Memory' by Udo Schilcher, Jorge F. Schmidt, Arke Vogell, and Christian Bettstetter<br>
http://data.bettstetter.com/publications/schilcher-2021-acsos-swarmalators.pdf

The model is extended by adding a momentum term to analyze whether this accelerates convergence.

### Imports

In [ ]:
from swarmalator_model.simulation import Simulation
from swarmalator_model.simulation_run import Simulation_run
from swarmalator_model.preset import Preset
from swarmalator_model.analysis import Analysis
from swarmalator_model import helper_functions as hlp
import os

### Run Simulation

Different patterns emerge when swarmalators converge depending on *J* and *K*.

|Pattern               |   J  |   K   |
|----------------------|------|-------|
|Static sync           |  0.1 |   1   |
|static async          |  0.1 |  -1   |
|static phase wave     |    1 |   0   |
|splintered phase wave |    1 | -0.1  |
|active phase wave     |    1 | -0.75 |

In [ ]:
sim = Simulation(
    plot_size=1000,
    logging=True,
    num_swarmalators=100,
    memory_init='random',
    coupling_probability=1.0,
    J=1,
    K=0,
    alpha=0.5)
sim.run_simulation()

### Automatic Simulation Runs

Load presets from JSON files created by the simulation tool.

In [ ]:
presets = []
for p in os.listdir('presets'):
    preset = Preset()
    preset.load_from_json(p)
    presets.append(preset)

presets

Automatically run simulations and save the resulting data for a given set of presets

In [ ]:
Simulation_run(presets=presets, sim_time=30).start()

### Analyze Simulation Data

Load datasets into Dataset objects

In [ ]:
file1 = os.listdir('sim_data')[0]
dataset1 = hlp.load_data('sim_data\\' + file1)
dataset1.summary()

#### Auto analyze data

Get all datasets

In [ ]:
all_datasets = []
for d in os.listdir('sim_data'):
    dataset = hlp.load_data('sim_data\\' + d)
    all_datasets.append(dataset)

Add datasets to an Analysis object

In [ ]:
a = Analysis()

# filter datasets where coupling probability = 1
datasets = [d for d in all_datasets if d.parameters['cp'] == 0.1]

# add datasets to analysis
for d in datasets:
    pattern = ''
    string = str(d.parameters['j']) + '_' + str(d.parameters['k'])
    match string:
        case '0.1_1.0':
            pattern = 'static sync'
        case '0.1_-1.0':
            pattern = 'static async'
        case '1.0_0.0':
            pattern = 'static phase wave'
        case '1.0_-0.1':
            pattern = 'splintered phase wave'
        case '1.0_-0.75':
            pattern = 'active phase wave'
        case _:
            pattern = 'unknown pattern'

    pattern += ' cp' + str(d.parameters['cp']) + ' a' + str(d.parameters['a'])
    a.add_dataset(d, pattern)

a.list_datasets()

Plot results

In [ ]:
ds_to_plot = list(a.datasets.keys())

# pairwise plot
i = 0
while i < len(ds_to_plot):
    a.plot_avg_speed([ds_to_plot[i], ds_to_plot[i + 1]], save=True)
    i += 2